In [3]:
pip install mysql-connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 571.4 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp311-cp311-macosx_11_0_arm64.whl size=247956 sha256=af4900a32a237f2de692574f916a001a7d71ec30b21f657d18609dcdad3e6820
  Stored in directory: /Users/jankuchli/Library/Caches/pip/wheels/17/cd/ed/2d49e9bac69cf09382e4c7cc20a2511202b48324b87db26019
Successfully built mysql-connector
Note: you may need to restart the kernel to use updated packages.


In [9]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

# MySQL connection parameters
host = '127.0.0.1'
port = '13306'
user = 'admin'
password = 'admin123'
database = 'immo'  
try:
    connection = mysql.connector.connect(host=host,
                                         port=port,
                                         user=user,
                                         password=password,
                                         database=database)

    if connection.is_connected():
        print("Connected to MySQL server")
        cursor = connection.cursor()

        query = """
        CREATE TABLE IF NOT EXISTS Location (
            LocationId VARCHAR(50) PRIMARY KEY,
            Street VARCHAR(150) NULL,
            ZIP INT NULL,
            longitude VARCHAR(50),
            latitude VARCHAR(50)
        );

        CREATE TABLE IF NOT EXISTS Property (
            ListingId VARCHAR(50) NULL,
            Timestamp DATETIME NULL,
            Rooms DECIMAL(5, 1) NULL,
            SquareMeter INT NULL,
            Floor VARCHAR(50) NULL,
            Availability VARCHAR(100) NULL,
            ObjectType VARCHAR(50) NULL,
            YearBuilt DATE NULL,
            Price DECIMAL(18, 2) NULL,
            AdditionalCost DECIMAL(18, 2) NULL,
            NetPrice DECIMAL(18, 2) NULL,
            LocationId VARCHAR(50) NULL,
            PropertyAdditionalFeaturesId VARCHAR(50) NULL,
            PropertyDescription VARCHAR(4000) NULL,
            Vendor VARCHAR(50) NULL,
            Canton VARCHAR(50) NULL,
            FOREIGN KEY (LocationId) REFERENCES Location(LocationId)
        );

        CREATE TABLE IF NOT EXISTS PropertyAdditionalFeatures (
            ListingId VARCHAR(50) NULL,
            Feature VARCHAR(250) NULL
        );
        """


        cursor.execute(query, multi=True)
        print("Tables created successfully")

except Error as e:
    print("Error connecting to MySQL:", e)

finally:
    if connection.is_connected():
        # Close cursor and connection
        cursor.close()
        connection.close()
        print("MySQL connection closed")

Connected to MySQL server
Tables created successfully
MySQL connection closed


In [21]:
import pandas as pd
import mysql.connector
import os
import numpy as np

# Connect to MySQL server
conn = mysql.connector.connect(
    host = '127.0.0.1',
    port = '13306',
    user = 'admin',
    password = 'admin123',
    database = 'immo'
)

# Go to correct directory:
# Get the current working directory
current_directory = os.getcwd()
# Go back one directory (up one level)
parent_directory = os.path.dirname(current_directory)
# Go into the "TablesDB" folder
tables_directory = os.path.join(parent_directory, "TablesDB")
# Change the current working directory to the "TablesDB" folder
os.chdir(tables_directory)


csv_file_path = 'Location_v2.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path, delimiter=",")
df = pd.DataFrame(df)
print(df.shape)
df = df[df['ZIP']>0]
print(df.shape)
df['ZIP'] = df['ZIP'].astype("int64")
df['longitude'] = df['longitude'].astype("string")
df['latitude'] = df['latitude'].astype("string")
df = df.replace({np.nan:None})

# Define the SQL statement for inserting data into the Location table
insert_query = """
INSERT INTO Location (LocationId, Street, ZIP, longitude, latitude)
VALUES (%s, %s, %s, %s, %s)
"""

# Create a cursor object
cursor = conn.cursor()

# Iterate over each row of the DataFrame and insert data into the Location table
for index, row in df.iterrows():
    # Extract data from the DataFrame row
    location_id = row['LocationId']
    street = row['Street']
    zip_code = row['ZIP']
    longitude = row['longitude']
    latitude = row['latitude']

    # Execute the SQL query to insert data into the Location table
    cursor.execute(insert_query, (location_id, street, zip_code, longitude, latitude))

# Commit changes and close the connection
conn.commit()
conn.close()


(11352, 5)
(11295, 5)


In [17]:
import os

print(os.getcwd())

# Get the current working directory
current_directory = os.getcwd()

# Go back one directory (up one level)
parent_directory = os.path.dirname(current_directory)

# Go into the "TablesDB" folder
tables_directory = os.path.join(parent_directory, "TablesDB")

# Change the current working directory to the "TablesDB" folder
os.chdir(tables_directory)

print(os.getcwd())


/Users/jankuchli/Documents/ZHAW/2024-FS/applied_data_science/ADS_Projekt/Immo-Projekt-DataScience/setupDB
/Users/jankuchli/Documents/ZHAW/2024-FS/applied_data_science/ADS_Projekt/Immo-Projekt-DataScience/TablesDB


In [25]:

# Get the current working directory
current_directory = os.getcwd()
# Go back one directory (up one level)
parent_directory = os.path.dirname(current_directory)
# Go into the "TablesDB" folder
tables_directory = os.path.join(parent_directory, "TablesDB")
# Change the current working directory to the "TablesDB" folder
os.chdir(tables_directory)


csv_file_path = 'Location_v2.csv'
# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path, delimiter=",")
filtered = df[df['LocationId'] == '030641ca-357e-45cc-adae-fe056bc9a13a']
filtered



,LocationId,Street,ZIP,longitude,latitude
9304,030641ca-357e-45cc-adae-fe056bc9a13a,Schützengraben 6,4051.0,NaN,NaN
